In [477]:
# import requests
import pandas as pd
import numpy as np
# import json
# import datetime as dt
# !pip install pandas==1.4


In [478]:
df = pd.read_csv('MunichAccessiblePoints_CleanBefore.csv',low_memory=False, decimal=';')
print(len(df))

13771


In [479]:
"""
DROP ROWS
"""
# df has 13770 rows at the initial 

# Drop rows with empty names, 11249 rows left
df = df.dropna(subset=['name'])

# Drop rows with duplicate names, 8511 rows left
df = df.drop_duplicates(subset=['name'])

# Drop 'wheelchait' == "no", 5553 rows left
df = df.drop(df.loc[df['wheelchair']== "no"].index)
print(len(df))

# Select certain columns
df = df[["Latitude", "Longitude","name","wheelchair","wheelchair:description",
           "amenity","opening_hours","tourism","atm","website","cuisine","toilets:wheelchair",
           "subway","tram","bus","railway","public_transport"]]

# Drop rows with "amenity == na and tourism == na", 2665 rows left
df = df.drop(df.loc[df['amenity'].isna() & df['tourism'].isna() & df['public_transport'].isna()].index)


5602


In [480]:
# sorted(df)
df["public_transport"].value_counts()

stop_position    443
station           48
platform          37
Name: public_transport, dtype: int64

In [481]:
""" 
Drop certain amentity types
"""
df = df.drop(df.loc[df['amenity']== "bicycle_rental"].index)
df = df.drop(df.loc[df['amenity']== "brothel"].index)
df = df.drop(df.loc[df['amenity']== "car_rental"].index)
df = df.drop(df.loc[df['amenity']== "car_sharing"].index)
df = df.drop(df.loc[df['amenity']== "car_wash"].index)
df = df.drop(df.loc[df['amenity']== "dentist"].index)
df = df.drop(df.loc[df['amenity']== "doctor"].index)
df = df.drop(df.loc[df['amenity']== "driving_school"].index)
df = df.drop(df.loc[df['amenity']== "fixme"].index)
df = df.drop(df.loc[df['amenity']== "fuel"].index)
df = df.drop(df.loc[df['amenity']== "kindergarten"].index)
df = df.drop(df.loc[df['amenity']== "gambling"].index)
df = df.drop(df.loc[df['amenity']== "school"].index)

In [482]:
"""
SORT
"""
# Sort all data based on the ascending names
df = df.sort_values(by=['name'])

In [483]:
"""
CHANGE the LAYOUT of VALUES
"""

df["amenity"] = df["amenity"].str.title()
df["tourism"] = df["tourism"].str.title()
df["cuisine"] = df["cuisine"].str.title()

# df= df.replace({"Post_Office": "Post Office", "Public_Bookcase": "Public Bookcase","Fast_Food": "Fast Food",
#                "Internet_Cafe": "Cafe"})

df['toilets:wheelchair'] = df['toilets:wheelchair'].replace({"yes":"Fully Wheelchair Accessible", "limited":"Partially Wheelchair Accessible"})


"""
Would show the bug: can only read 'str' value
if we don't put [decimal=';'] into [read_csv]
"""
df = df.apply(lambda x: x.str.replace(';',',')) # Replace all ';' into ',', so filter function can read them 
df = df.apply(lambda x: x.str.replace('_',' ')) # Replace all '_' into ',', so filter function can read them 


In [484]:
df['subway'] = df['subway'].replace({"yes":"Subway Station"})
df['tram'] = df['tram'].replace({"yes":"Tram Stop"})
df['bus'] = df['bus'].replace({"yes":"Bus Stop"})
df['railway'] = df['railway'].replace({"station":"Train Station", "tram stop":np.nan, "stop":np.nan, "halt":np.nan})

# Merge values in each public traffic related column into a new column and seperate them by comma ','
df['PublicTraffic'] = df[['subway', 'tram', 'bus', 'railway']].apply(lambda x: ' ,'.join(x[x.notnull()]), axis = 1)
df['PublicTraffic'].replace('', np.nan, inplace=True)

In [485]:
df["tourism"].value_counts()
df['tourism'] = df['tourism'].replace({"Information":"Tourist Information Center"})

df['amenity'] = df['amenity'].replace({"Bus Station":np.nan})

In [486]:
# Merge values in each public traffic related column into a new column and seperate them by comma ','
df['What'] = df[['amenity', 'tourism','PublicTraffic']].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)
print(len(df))

2993


In [487]:
df = df.drop(df.loc[df['What']== np.nan].index)
print(len(df))

2993


In [493]:
"""
Pandas will recognise a value as null if it is a np.nan object, which will print as NaN in the DataFrame. 
Your missing values are probably empty strings, which Pandas doesn't recognise as null. 
To fix this, you can convert the empty stings (or whatever is in your empty cells) to np.nan objects using replace(), 
and then call dropna()on your DataFrame to delete rows with null tenants.
"""

df['What'].replace('', np.nan, inplace=True)
df.dropna(subset=['What'], inplace=True)

df = df.apply(lambda x: x.str.replace('Of ','of ')) 
df = df.apply(lambda x: x.str.replace('Atm ','ATM')) 
print(len(df))

2968


In [494]:
df["amenity"].value_counts()

Restaurant           751
Cafe                 256
Fast Food            227
Pharmacy             225
Doctors              152
Pub                   81
Bar                   65
Bank                  40
Social Facility       37
Ice Cream             35
Library               33
Post Office           32
Parking               25
Cinema                23
Place of Worship      23
Community Centre      22
Biergarten            17
Veterinary            16
Hospital              16
Marketplace           15
Arts Centre           14
Theatre               14
Nightclub             12
Atm                   11
Fountain              10
Clinic                 9
University             9
Police                 7
Toilets                6
Parking Entrance       5
Vending Machine        4
Childcare              4
College                3
Canteen                3
Townhall               3
Drinking Water         3
Events Venue           3
Taxi                   2
Internet Cafe          2
Courthouse             2


In [495]:
pd.set_option("display.max_rows", None)
# print(df["cuisine"].value_counts())

In [496]:
# df1 = df["wheelchair"].str.title()

In [497]:
# Save pandas data as a new CSV file

df.to_csv('points.csv',header=True)